In [1]:

import hist
import dask
import awkward as ak
import hist.dask as hda
import dask_awkward as dak
import numpy as np

from coffea import processor
from coffea.nanoevents.methods import candidate
from coffea.nanoevents.methods import vector as vec
from coffea.dataset_tools import (
    apply_to_fileset,
    max_chunks,
    preprocess,
)
from distributed import Client
client = Client("tls://localhost:8786")

def match_b_to_a(a_obj, b_obj, match_dr):
    '''this function will find the match the closest b to a 
        match_dr is a rough cut, it set a limit; in case i got the clossest, but they are already far enough to consider to be different things'''
    a = ak.zip(
        {"pt": ak.fill_none(a_obj.pt, 0.0),
         "eta": ak.fill_none(a_obj.eta, 0.0),
         "phi": ak.fill_none(a_obj.phi, 0.0),
         "mass": ak.fill_none(getattr(a_obj, "mass", ak.zeros_like(a_obj.pt)), 0.0)},
        with_name="PtEtaPhiMCandidate", behavior=candidate.behavior,
    )
    b = ak.zip(
        {"pt": ak.fill_none(b_obj.pt, 0.0),
         "eta": ak.fill_none(b_obj.eta, 0.0),
         "phi": ak.fill_none(b_obj.phi, 0.0),
         "mass": ak.fill_none(getattr(b_obj, "mass", ak.zeros_like(b_obj.pt)), 0.0)},
        with_name="PtEtaPhiMCandidate", behavior=candidate.behavior,
    )

    # Cartesian pairs per event: shape [evt, len(a_i), len(b_i)]
    pairs = ak.cartesian({"a": a, "b": b}, axis=1, nested=True)

    # delta R for every (a,b)
    dr = pairs.a.delta_r(pairs.b)
    
    
    #dr = a_obj.delta_r(b_obj)
    # get things within the match_dr, and keep the shape by leaving a default value
    default = 99999.0
    dr_incone = ak.where(dr < match_dr, dr, default) 
    # get the closest b for each a
    min_dr = ak.min(dr_incone, axis=-1, initial=default)  # note if no min is found, assign the default value. this var is to check for argmin          
    # make sure the code will work even if there are no b or all are cut out of the cone
    argmin = ak.argmin(dr_incone, axis=-1) 
    valid = (min_dr < default)              
    argmin_safe = ak.where(valid, argmin, 0)

    b_works  = b_obj[argmin_safe]                          
    a_match = a_obj[valid]                                
    b_match = b_works[valid]
    dr_match = min_dr[valid]


    #return a_out, b_out, dr_out
    return a_match, b_match, dr_match 

    

class MyProcessor(processor.ProcessorABC):
    def __init__(self):
        pass

    def process(self, events):
        GEN_FLAGS = ["isLastCopy"]
        #GEN_FLAGS1 = ["fromHardProcess", "isLastCopy"]
        dataset = events.metadata['dataset']
        # getting gen level info for b and e
        #print("start")
        gen_part = events.GenPart
        gen_t_temp = gen_part[(gen_part.pdgId == 6)]
        gen_t = gen_t_temp[(gen_t_temp.hasFlags(GEN_FLAGS))]

        
        #--------------------check------------------------------
        n_child_raw   = ak.fill_none(ak.num(gen_t.children,          axis=2), 0)
        print("tops per evt (first 100):", ak.to_list(ak.num(gen_t, axis=1)[:100]))
        print("n children per top (raw)   first 100 evts:", ak.to_list(n_child_raw[:100]))
        t_children_pdg = gen_t.children.pdgId
        print("top.children pdgId (first 30 evts):")
        print(ak.to_list(t_children_pdg[:30]))
        #---------------------------------------------------------



        #print("check1")
        gen_t_child = gen_t.children
        gen_b = gen_t_child[(gen_t_child.pdgId == 5)]



        gen_w_lastcopy = gen_part[(gen_part.pdgId == 24) & (gen_part.hasFlags(GEN_FLAGS))]
        gen_w_child = gen_w_lastcopy.children
        gen_e = gen_w_child[(gen_w_child.pdgId == -11)]


        #print("check2") 
        
        #print("1")

        n_b_per_t = ak.fill_none(ak.num(gen_b,          axis=2), 0)  
        n_e_per_t = ak.fill_none(ak.num(gen_e, axis=2), 0) 
        n_top     = ak.fill_none(ak.num(gen_t,          axis=1), 0) 
        
        #-------------need to chage for different samples 
        good_b_evt = ak.fill_none(ak.any(n_b_per_t >= 1, axis=1), False) 
        good_e_evt = ak.fill_none(ak.any(n_e_per_t >= 1, axis=1), False)
        event_good = (n_top >= 1) & good_b_evt & good_e_evt

        events_sel = events[event_good]  
        # cut from the sliced events to make sure the lists are clean:
        gen_part = events_sel.GenPart
        gen_t    = gen_part[(gen_part.pdgId == 6) & (gen_part.hasFlags(GEN_FLAGS))]
        gen_b = gen_t.children[(gen_t.children.pdgId == 5)]
        
        gen_w_lastcopy = gen_part[(gen_part.pdgId == 24) & (gen_part.hasFlags(GEN_FLAGS))]
        gen_w_child = gen_w_lastcopy.children
        gen_e = gen_w_child[(gen_w_child.pdgId == -11)] 
        
        
        e_all = events_sel.Electron
        b_jet = events_sel.Jet

        gen_e_all = ak.flatten(gen_e, axis=2)
        gen_b_all = ak.flatten(gen_b, axis=2) 
        #print("2")




        
        # get gen delta R
        gen_dR = gen_e_all.delta_r(gen_b_all)
        #print("3")

        e_gen_match, e_rec_match, dr_e = match_b_to_a(gen_e_all, e_all, match_dr=0.1)

        
        #print("4")
        b_gen_match, b_reco_match, dr_b = match_b_to_a(gen_b_all, b_jet, match_dr=0.1)
        #print("5")


        
        # --- sanity check ---
        nevt_evt  = dak.num(events_sel, axis=0).compute()
        nevt_e    = dak.num(gen_e_all,      axis=0).compute()
        nevt_b    = dak.num(gen_b_all,      axis=0).compute()
        nevt_jet  = dak.num(b_jet,      axis=0).compute()
        nevt_reco = dak.num(e_all,      axis=0).compute()
        print(f"[events] sel={nevt_evt}, gen_e={nevt_e}, gen_b={nevt_b}, reco_e={nevt_reco}, reco_j={nevt_jet}")
        
        
        print("events_sel (axis0):", dak.num(events_sel, axis=0).compute())
        print("gen_b     (axis0):",  dak.num(gen_b,     axis=0).compute())      
        print("gen_b_all (axis0):",  dak.num(gen_b_all, axis=0).compute())     
        
        print("n_b per event (first 10):", ak.to_list(ak.num(gen_b_all, axis=1)[:10]))
        print("next")
        print("n tops/evt:", ak.to_list(ak.num(gen_t, axis=1)[:10]))
        print("n b/evt:",        ak.to_list(ak.num(gen_b_all, axis=1)[:10]))
        
        #print("n W/evt:",    ak.to_list(ak.num(gen_w, axis=2)[:10]))  
        print("n e/evt:",        ak.to_list(ak.num(gen_e_all, axis=1)[:10]))
        
        
        print("before flattening list")
        nevt_evt  = dak.num(events_sel, axis=0).compute()
        #nevt_e    = dak.num(gen_e,      axis=0).compute()
        nevt_b    = dak.num(gen_b,      axis=0).compute()
        nevt_jet  = dak.num(b_jet,      axis=0).compute()
        nevt_reco = dak.num(e_all,      axis=0).compute()
        #print(f"[events] sel={nevt_evt}, gen_e={nevt_e}, gen_b={nevt_b}, reco_e={nevt_reco}, reco_j={nevt_jet}")
        print(f"[events] sel={nevt_evt}, gen_b={nevt_b}, reco_e={nevt_reco}, reco_j={nevt_jet}")
        #-----------------------------------------------------


        return {
            "e_gen_match": e_gen_match,
            "e_rec_match": e_rec_match,
            "dr_e": dr_e,
            "b_gen_match": b_gen_match,
            "b_reco_match": b_reco_match,
            "dr_b": dr_b,
    
            # diagnostics (post-selection)
            "n_top": ak.num(gen_t, axis=1),  # [evt]
            "n_b_evt": ak.num(gen_b_all, axis=1),
            "n_e_evt": ak.num(gen_e_all, axis=1),
            "n_e_reco_sel": ak.num(e_all, axis=1),
            "n_j_reco_sel": ak.num(b_jet, axis=1),
    }
    
    
    def postprocess(self, accumulator):
        pass


/usr/local/lib/python3.12/site-packages/coffea/nanoevents/schemas/fcc.py:5: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector


In [2]:
from coffea.nanoevents import NanoAODSchema
from coffea.dataset_tools import (
    apply_to_fileset,
    max_chunks,
    preprocess,
)
fileset = {
    'Suu-tu_MC': {
        "files": {
            'root://cms-xrd-global.cern.ch//store/mc/RunIISummer20UL18NanoAODv9/SuuToTU_TToBLNu_MSuu-9000_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/2550000/37390A7B-151F-FA45-BC3E-17F59C1D6637.root': "Events",
            #'root://xcache//store/mc/RunIII2024Summer24NanoAODv15/DYJetsToLL_M-50_TuneCP5_13p6TeV-madgraphMLM-pythia8/NANOAODSIM/Pilot2024wmLHEGS_150X_mcRun3_2024_realistic_v1-v2/2540000/8908b7ad-b7b7-400c-8c4d-a859666fe8e2.root':"Events",
        }
    }
}


dataset_runnable, dataset_updated = preprocess(
    fileset,
    align_clusters=False,
    step_size=100_000,
    files_per_batch=1,
    skip_bad_files=True,
    save_form=False,
)

to_compute = apply_to_fileset(
                MyProcessor(),
                dataset_runnable,
                schemaclass=NanoAODSchema,
            )
(out,) = dask.compute(to_compute)
#(out,) = dask.compute(to_compute, scheduler="synchronous")
ds = out["Suu-tu_MC"]  

/usr/local/lib/python3.12/site-packages/coffea/nanoevents/schemas/nanoaod.py:264: RuntimeWarning: Missing cross-reference index for LowPtElectron_electronIdx => Electron
  warnings.warn(
/usr/local/lib/python3.12/site-packages/coffea/nanoevents/schemas/nanoaod.py:264: RuntimeWarning: Missing cross-reference index for LowPtElectron_photonIdx => Photon
  warnings.warn(


tops per evt (first 100): [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
n children per top (raw)   first 100 evts: [[2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2]]
top.children pdgId (first 30 evts):
[[[24, 5]], [[24, 5]], [[24, 5]], [[24, 5]], [[24, 5]], [[24, 5]], [[24, 5]], [[24, 5]], [[24

In [3]:
n_e_matched = ak.num(ds["e_gen_match"], axis=1)   # [n_evt]
n_b_matched = ak.num(ds["b_gen_match"], axis=1)   # [n_evt]
print("Total events:", len(n_e_matched))
print("Events with ≥1 matched e:", int(ak.sum(n_e_matched > 0)))
print("Events with ≥1 matched b:", int(ak.sum(n_b_matched > 0)))
print("Events with ≥1 matched e AND ≥1 matched b:",
      int(ak.sum((n_e_matched > 0) & (n_b_matched > 0))))

Total events: 2315
Events with ≥1 matched e: 2114
Events with ≥1 matched b: 1425
Events with ≥1 matched e AND ≥1 matched b: 1268
